# Entrenamiento y prueba de un modelo clasificador basado en redes neuronales sencillas

Si se ha realizado el notebook del árbol de decisión (cosa que se recomienda hacer antes de estudiar este), se habrá visto que el modelo obtenido no es realmente efectivo para el problema que enfrentamos. No nos permite hacer predicciones fiables sobre si es recomendable subir o no a los Lagos de Covadonga. 

Es por ello que vamos a probar con un modelo algo más complejo, dado que el anterior modelo nos salía algo subajustado debido a la escasa cantidad de datos. Aún así, como el problema es simple, tendremos que mantener un equilibrio en la complejidad. Es por ello que vamos a probar con una red neuronal no muy densa, que nos permita obtener mejores predicciones que los árboles de decisión, teniendo cuidado de que no se produzca sobreajuste.

Para el planteamiento de un modelo con redes neuronales, emplearemos la librería `scikit-learn` usada antes, pero también introduciremos la librería `keras` para hacer lo mismo de otra forma, con fines educativos.

Como nos encontramos el mismo problema que antes en la división del dataset tan pequeño en *training* y *test*, procuraremos emplear siempre que podamos técnicas de validación cruzada.

## Paquetes

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import LeaveOneOut

## Importación de datos y tratamiento

Vamos a seguir el mismo procedimiento que en el otro notebook. Primero, descargamos los datos de los CSV's generados con las llamadas a la API de la AEMET.

In [ ]:
semilla = 123
np.random.seed(semilla)
base_dir = os.path.dirname(os.path.abspath(os.getcwd()))

datos = pd.read_csv(os.path.join(base_dir, "data", "processed", "data.csv"), sep=";")
datos_sin_etiquetar = pd.read_csv(os.path.join(base_dir, "data", "processed", "data_sin_etiquetar.csv"), sep=";")

Una vez cargados, debemos convertir las variables cualitativas a *dummies* para asegurar un correcto entrenamiento y uso del modelo. Lo haremos con la función `get_dummies` de `pandas`.

Primero, extraemos las variable categóricas de los dataset:

In [ ]:
categoricas = datos.select_dtypes(include= ["object", "category"]).columns
categoricas_sin_etiquetar = datos_sin_etiquetar.select_dtypes(include= ["object", "category"]).columns
categoricas

Index(['fecha', 'altonubes', 'nubosidad', 'lluvia', 'viento', 'SUBIR'], dtype='object')

No vamos a tener en cuenta las variables "fecha", dado que esta es el índice (clave primaria que identifica cada dato); ni "SUBIR", dado que es el target a predecir y no haría falta transformarlo. De hecho, la variable "fecha" podrá eliminarse por completo del dataset, dado que no influye a la hora de realizar las predicciones.

In [ ]:
categoricas = categoricas.delete([0, 5])
datos.drop("fecha", axis=1, inplace=True)
categoricas

Index(['altonubes', 'nubosidad', 'lluvia', 'viento'], dtype='object')

De las "sin etiquetar", vamos a eliminar únicamente la fecha.

In [ ]:
datos_sin_etiquetar.drop("fecha", axis=1, inplace=True)
categoricas_sin_etiquetar = categoricas_sin_etiquetar.delete(0)
categoricas_sin_etiquetar

Index(['altonubes', 'nubosidad', 'lluvia', 'viento'], dtype='object')

Transformamos a dummies y eliminamos las variables originales:

In [ ]:
categoricas_dummies = pd.get_dummies(datos[categoricas], drop_first = True, dummy_na = True)
categoricas_sin_etiquetar_dummies = pd.get_dummies(datos_sin_etiquetar[categoricas_sin_etiquetar], drop_first = True, dummy_na = True)

datos.drop(categoricas, axis = 1, inplace = True)   # inplace para que se elimine sobre el dataset
datos_sin_etiquetar = datos_sin_etiquetar.reindex(columns=datos.drop(columns="SUBIR").columns, fill_value=0)

# Tras eliminar los atributos originales, concatenamos los nuevos atributos creados para las variables categóricas.
datos = pd.concat([datos, categoricas_dummies], axis = 1)
datos_sin_etiquetar = pd.concat([datos_sin_etiquetar, categoricas_sin_etiquetar_dummies], axis = 1)

# Mostramos cómo queda tras este proceso
datos.head()

,prec,tmin,tmax,velmedia,hrMedia,hrMax,hrMin,SUBIR,altonubes_mid,altonubes_nan,nubosidad_escasa,nubosidad_media,nubosidad_nan,lluvia_no,lluvia_posible,lluvia_nan,viento_fuerte,viento_moderado,viento_nan
0,0.0,11.9,20.4,1.4,99.0,100.0,61.0,NO,False,False,False,False,False,False,True,False,False,True,False
1,2.0,11.7,25.0,2.5,64.0,100.0,40.0,NO,False,True,False,False,True,False,False,True,False,False,True
2,23.8,13.7,26.6,5.6,54.0,94.0,34.0,SI,True,False,True,False,False,False,False,False,True,False,False
3,2.0,11.5,19.9,3.6,86.0,100.0,50.0,SI,False,True,False,False,False,False,False,False,False,True,False
4,0.0,11.4,18.2,1.4,95.0,100.0,80.0,NO,False,False,True,False,False,True,False,False,False,False,False
